## 1. Import package & Dataset

- DataSource: The world Happiness Report : https://worldhappiness.report/

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
all_happiness_data = pd.read_csv("world-happiness-report-raw.csv")
all_happiness_data

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268
...,...,...,...,...,...,...,...,...,...,...,...
1944,Zimbabwe,2016,3.735,7.984,0.768,54.40,0.733,-0.095,0.724,0.738,0.209
1945,Zimbabwe,2017,3.638,8.016,0.754,55.00,0.753,-0.098,0.751,0.806,0.224
1946,Zimbabwe,2018,3.616,8.049,0.775,55.60,0.763,-0.068,0.844,0.710,0.212
1947,Zimbabwe,2019,2.694,7.950,0.759,56.20,0.632,-0.064,0.831,0.716,0.235


## 2. Get data ready

- Clean Data

In [4]:
all_happiness_data.count()

Country name                        1949
year                                1949
Life Ladder                         1949
Log GDP per capita                  1913
Social support                      1936
Healthy life expectancy at birth    1894
Freedom to make life choices        1917
Generosity                          1860
Perceptions of corruption           1839
Positive affect                     1927
Negative affect                     1933
dtype: int64

### 2-1 Overview

- How Many Country Are there?

In [5]:
n = len(pd.unique(all_happiness_data['Country name']))
print("Number of unique Countries :", n)

Number of unique Countries : 166


- How Many Years Are there?

In [6]:
n = len(pd.unique(all_happiness_data['year']))
print("Number of unique year :", n)

Number of unique year : 16


- We have total of 1949 rows
- However 166(Countries) * 16(year) = 2656
- Means we have some missing years within some countries, let's identify them.

- Is the year for the countries align??

In [7]:
# sort dataset by country & year
all_happiness_data.sort_values(by=['Country name', 'year'])
all_happiness_data.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [8]:
all_happiness_data['year'].value_counts().sort_index()

2005     27
2006     89
2007    102
2008    110
2009    114
2010    124
2011    146
2012    142
2013    137
2014    145
2015    143
2016    142
2017    147
2018    142
2019    144
2020     95
Name: year, dtype: int64

- As most of the county have data from 2007 to 2020. I want to elimiate the data from 2005 & 2006

In [9]:
all_happiness_data = all_happiness_data[all_happiness_data['year'] > 2006]

In [10]:
# let's check
all_happiness_data['year'].value_counts().sort_index()

2007    102
2008    110
2009    114
2010    124
2011    146
2012    142
2013    137
2014    145
2015    143
2016    142
2017    147
2018    142
2019    144
2020     95
Name: year, dtype: int64

### 2-2 Deal with missing values

In [11]:
# Check the null count
print('Count of null values by column:')
all_happiness_data.isna().sum()

Count of null values by column:


Country name                          0
year                                  0
Life Ladder                           0
Log GDP per capita                   35
Social support                       13
Healthy life expectancy at birth     54
Freedom to make life choices         27
Generosity                           55
Perceptions of corruption           102
Positive affect                      21
Negative affect                      14
dtype: int64

### 2-2-1 Set up approach for each column to deal with missing values

- As all of our missing column is numerical, fill with the average within the country will be fair

In [12]:
mean_df = all_happiness_data.groupby('Country name', as_index = False).mean()

In [13]:
for index, row in all_happiness_data.iterrows():

        if(pd.isnull(row['Log GDP per capita'])):
            all_happiness_data.at[index , 'Log GDP per capita'] = mean_df.loc[mean_df['Country name'] == row['Country name']]['Log GDP per capita']
        if(pd.isnull(row['Social support'])):
            all_happiness_data.at[index , 'Social support'] = mean_df.loc[mean_df['Country name'] == row['Country name']]['Social support']
        if(pd.isnull(row['Healthy life expectancy at birth'])):
            all_happiness_data.at[index , 'Healthy life expectancy at birth'] = mean_df.loc[mean_df['Country name'] == row['Country name']]['Healthy life expectancy at birth']
        if(pd.isnull(row['Freedom to make life choices'])):    
            all_happiness_data.at[index , 'Freedom to make life choices'] = mean_df.loc[mean_df['Country name'] == row['Country name']]['Freedom to make life choices']
        if(pd.isnull(row['Generosity'])):    
            all_happiness_data.at[index , 'Generosity'] = mean_df.loc[mean_df['Country name'] == row['Country name']]['Generosity']
        if(pd.isnull(row['Perceptions of corruption'])):    
            all_happiness_data.at[index , 'Perceptions of corruption'] = mean_df.loc[mean_df['Country name'] == row['Country name']]['Perceptions of corruption']
        if(pd.isnull(row['Positive affect'])):    
            all_happiness_data.at[index , 'Positive affect'] = mean_df.loc[mean_df['Country name'] == row['Country name']]['Positive affect']
        if(pd.isnull(row['Negative affect'])):    
            all_happiness_data.at[index , 'Negative affect'] = mean_df.loc[mean_df['Country name'] == row['Country name']]['Negative affect']

In [14]:
# Check the null count
print('Count of null values by column:')
all_happiness_data.isna().sum()

Count of null values by column:


Country name                         0
year                                 0
Life Ladder                          0
Log GDP per capita                  18
Social support                       1
Healthy life expectancy at birth    35
Freedom to make life choices         0
Generosity                          18
Perceptions of corruption           26
Positive affect                      2
Negative affect                      1
dtype: int64

- There are still some missing data, and I will like to fill them with average of the column as I don't want to delet them to loss any more data

In [15]:
all_happiness_data.fillna(all_happiness_data.mean(), inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [16]:
# Check the null count
print('Count of null values by column:')
all_happiness_data.isna().sum()

Count of null values by column:


Country name                        0
year                                0
Life Ladder                         0
Log GDP per capita                  0
Social support                      0
Healthy life expectancy at birth    0
Freedom to make life choices        0
Generosity                          0
Perceptions of corruption           0
Positive affect                     0
Negative affect                     0
dtype: int64

### 2-3 Add Continent Column

- As I will also like to analysis countries based on Continent, I will like to add this column to the all_happiness_data
- Find country to continent on Kaggle: https://www.kaggle.com/statchaitya/country-to-continent

In [17]:
continent_data = pd.read_csv("countryContinent.csv",encoding='ISO-8859-1')
continent_data

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0
...,...,...,...,...,...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,9.0,61.0
245,Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,2.0,15.0
246,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,142.0,145.0
247,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,Africa,Eastern Africa,2.0,14.0


### 2-3-1 Clean continent_data as I only need country name & continent & sub_region

In [18]:
continent_data = continent_data[['country', 'continent', 'sub_region']]
continent_data.rename(columns={'country':'Country name'}, inplace=True)
continent_data

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Country name,continent,sub_region
0,Afghanistan,Asia,Southern Asia
1,Åland Islands,Europe,Northern Europe
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,American Samoa,Oceania,Polynesia
...,...,...,...
244,Wallis and Futuna,Oceania,Polynesia
245,Western Sahara,Africa,Northern Africa
246,Yemen,Asia,Western Asia
247,Zambia,Africa,Eastern Africa


In [19]:
all_happiness_data = pd.merge(all_happiness_data, continent_data, on="Country name")

In [20]:
all_happiness_data

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,continent,sub_region
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258,Asia,Southern Asia
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237,Asia,Southern Asia
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275,Asia,Southern Asia
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267,Asia,Southern Asia
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268,Asia,Southern Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577,Zimbabwe,2016,3.735,7.984,0.768,54.40,0.733,-0.095,0.724,0.738,0.209,Africa,Eastern Africa
1578,Zimbabwe,2017,3.638,8.016,0.754,55.00,0.753,-0.098,0.751,0.806,0.224,Africa,Eastern Africa
1579,Zimbabwe,2018,3.616,8.049,0.775,55.60,0.763,-0.068,0.844,0.710,0.212,Africa,Eastern Africa
1580,Zimbabwe,2019,2.694,7.950,0.759,56.20,0.632,-0.064,0.831,0.716,0.235,Africa,Eastern Africa


In [21]:
# Check the null count
print('Count of null values by column:')
all_happiness_data.isna().sum()

Count of null values by column:


Country name                        0
year                                0
Life Ladder                         0
Log GDP per capita                  0
Social support                      0
Healthy life expectancy at birth    0
Freedom to make life choices        0
Generosity                          0
Perceptions of corruption           0
Positive affect                     0
Negative affect                     0
continent                           0
sub_region                          0
dtype: int64

## 3 Save csv

In [22]:
all_happiness_data.to_csv("world-happiness-report-new.csv", index=False)